# Formula Exploration Notebook

This notebook holds the attempt to explore how we could operate formulas in spreadsheet cell values.

## Known formula parsing in Sympy 

In [2]:
# Expanding cell ranges

from openpyxl.worksheet.datavalidation import expand_cell_ranges

sample_cell_range = "B2:B5"
sample_expanded_cells = expand_cell_ranges(sample_cell_range)
print(f"Expanded cell list from range of \"{sample_cell_range}\": {sample_expanded_cells}")

Expanded cell list from range of "B2:B5": {'B2', 'B4', 'B3', 'B5'}


In [3]:
# Parse the formula

sample_expanded_formula = "+".join(sample_expanded_cells)
print(sample_expanded_formula)

# We can use the sample_expanded_formula for comparison!

B2+B4+B3+B5


In [4]:
# Now, what about parsing / tokenizing the formulas?

from openpyxl.worksheet.datavalidation import expand_cell_ranges
from openpyxl.formula import Tokenizer
from openpyxl.formula.tokenizer import Token

def simplify_sum_range(cell_range):
    """
    This method will simplify the range in a SUM function so it could be parsed by Sympy.
    """
    expanded_cells = expand_cell_ranges(cell_range)
    return "+".join(expanded_cells)

def simplify_sum_range_list(cell_range_list):
    """
    This method will simplify the passed list of cell ranges in a SUM function so it could be parsed by Sympy.
    """
    
    print(f"Input: {cell_range_list}")
    if len(cell_range_list) == 0:
        return ""
    
    simplified_formula = simplify_sum_range(cell_range_list[0])
    print(f"Current simplified formula: {simplified_formula}")
    
    for cell_range in cell_range_list[1:]:
        current_formula = simplify_sum_range(cell_range)
        simplified_formula += f"+{current_formula}"
        
    print(f"Final simplified formula: {simplified_formula}")
    return simplified_formula

def simplify_formula(formula):
    """
    Returns simplified version of excel formula, if any.
    """
    # TODO: This only handles SUM for now.
    formula_tokenizer = Tokenizer(formula)
    pending_function = []
    item_buffer = []
    
    print(f"Tokenized form of formula {formula}:")
    for token in formula_tokenizer.items:
        print(token)
        if token.type == Token.FUNC and token.subtype == Token.OPEN:
            pending_function.append(token)
            
        elif token.type == Token.FUNC and token.subtype == Token.CLOSE:
            current_function = pending_function.pop()
            ranges = [item.value for item in item_buffer]
            return simplify_function_and_data(current_function.value, ranges)
        
        elif token.type != Token.SEP:
            item_buffer.append(token)
    
    # TODO: Map this out with proper case later.
    return formula.replace("=","")

def simplify_function_and_data(function, data):
    """
    Simplifies the passed Excel function string with the passed data list.
    """
    
    print(f"Requested function to be simplified: {function}")
    print(f"Requested data to be simplified: {data}")
    
    function = function.replace("(", "")
    # TODO: Add other functions here besides the SUM.
    if function == "SUM":
        return simplify_sum_range_list(data)
    
    # TODO: Revisit what to do if there's no matching function
    return data
    
    

In [5]:
# Test the method above

from sympy.parsing.sympy_parser import parse_expr
from sympy import simplify

formula_1 = "=SUM(B2:B5)"
formula_2 = "=B2+B3+B4+B5"

formula_1_simplified = simplify_formula(formula_1)
formula_2_simplified = simplify_formula(formula_2)

print(f"Simplified version of formula_1: {formula_1_simplified}")
print(f"Simplified version of formula_2: {formula_2_simplified}")

# Compare them using Sympy

formula_1_parsed = parse_expr(formula_1_simplified)
formula_2_parsed = parse_expr(formula_2_simplified)

print(f"Parsed formula_1's type: {type(formula_1_parsed)}, content: {formula_1_parsed}")
print(f"Parsed formula_2's type: {type(formula_2_parsed)}, content: {formula_2_parsed}")

is_equal_formula = simplify(formula_1_parsed - formula_2_parsed) == 0

print(f"Are the formulas equal: {is_equal_formula}")

Tokenized form of formula =SUM(B2:B5):
FUNC OPEN SUM(:
OPERAND RANGE B2:B5:
FUNC CLOSE ):
Requested function to be simplified: SUM(
Requested data to be simplified: ['B2:B5']
Input: ['B2:B5']
Current simplified formula: B2+B4+B3+B5
Final simplified formula: B2+B4+B3+B5
Tokenized form of formula =B2+B3+B4+B5:
OPERAND RANGE B2:
OPERATOR-INFIX  +:
OPERAND RANGE B3:
OPERATOR-INFIX  +:
OPERAND RANGE B4:
OPERATOR-INFIX  +:
OPERAND RANGE B5:
Simplified version of formula_1: B2+B4+B3+B5
Simplified version of formula_2: B2+B3+B4+B5
Parsed formula_1's type: <class 'sympy.core.add.Add'>, content: B2 + B3 + B4 + B5
Parsed formula_2's type: <class 'sympy.core.add.Add'>, content: B2 + B3 + B4 + B5
Are the formulas equal: True


## Unknown formula parsing in Sympy

Let's see whether sympy are able to handle unknown functions!

In [6]:
# Parsing SUM with similar variable location

from sympy.parsing.sympy_parser import parse_expr
from sympy import simplify

sum_formula_sample = "SUM(B2, B3, B4, B5)"
sum_formula_sample_parsed = parse_expr(sum_formula_sample)

print(f"Parsed formula: {sum_formula_sample_parsed}")

# Can we simplify this?
is_equal_formula = simplify(sum_formula_sample_parsed - sum_formula_sample_parsed) == 0
print(f"Is formula equal: {is_equal_formula}")

Parsed formula: SUM(B2, B3, B4, B5)
Is formula equal: True


In [7]:
# Since this works, what about parsing the SUM in form of range (e.g., B2:B5)?

sum_range_sample = "SUM(B2:B5)"
sum_range_sample_parsed = parse_expr(sum_range_sample)

print(f"Parsed formula: {sum_range_sample_parsed}")

# No we can't, it will identify the : as invalid syntax.


SyntaxError: invalid syntax (<string>, line 1)

In [9]:
# What about parsing unknown formula with variables in different position?

unknown_formula_1 = "UNKNOWN(A1, B2, C3)"
unknown_formula_2 = "UNKNOWN(A1, C3, B2)"

unknown_formula_1_parsed = parse_expr(unknown_formula_1)
unknown_formula_2_parsed = parse_expr(unknown_formula_2)

print(f"Parsed first unknown formula: {unknown_formula_1_parsed}")
print(f"Parsed second unknown formula: {unknown_formula_2_parsed}")

is_equal_formula = simplify(unknown_formula_1_parsed - unknown_formula_2_parsed) == 0
print(f"Is formula equal: {is_equal_formula}")

# Of course it's false.

Parsed first unknown formula: UNKNOWN(A1, B2, C3)
Parsed second unknown formula: UNKNOWN(A1, C3, B2)
Is formula equal: False


In [10]:
# Hol' up. What about nested formulas?

nested_unknown_formula = "SUM(SUM(A1, B1), SUM(A2, B2))"

nested_unknown_formula_parsed = parse_expr(nested_unknown_formula)

print(f"Parsed nested unknown formula: {nested_unknown_formula_parsed}")

# Can we simplify it?

is_equal_formula = simplify(nested_unknown_formula_parsed - nested_unknown_formula_parsed) == 0
print(f"Is formula equal: {is_equal_formula}")

Parsed nested unknown formula: SUM(SUM(A1, B1), SUM(A2, B2))
Is formula equal: True


## Range parsing and expanding

What kind of ranges that can be parsed and expanded by openpyxl?

In [11]:
# Different range types:

from openpyxl.worksheet.datavalidation import expand_cell_ranges

# Turns out Sheet won't work as an expanded range. Hmm.
# sample_ranges = ["B2:B5", "$A$1", "'Sheet 2'!B1"]

# However, expand_cell_ranges can take the dollars and change it to normal cell reference.
sample_ranges = ["B2:B5", "$A$1"]
expanded_ranges = [expand_cell_ranges(r) for r in sample_ranges]

print("Sample ranges:")
print(sample_ranges)
print("\nExpanded ranges:")
print(expanded_ranges)

Sample ranges:
['B2:B5', '$A$1']

Expanded ranges:
[{'B2', 'B4', 'B3', 'B5'}, {'A1'}]


## Conclusion

So far... here's what we can do:

1. Tokenize the input formula.
2. For each range found, expand the range and join it using comma (,).
3. Rebuild the formula so the input will be the formulas with commas.
4. Encode every letter to prevent clash with single letters that preserved for math (e.g. Euler's number, E)
    - Provide the decoding method too.

Issues to be tackled:
1. How do we handle functions with different cell references? The unit test?

In [23]:
# Functions in conclusion of the experiment above.

from openpyxl.formula import Tokenizer
from openpyxl.formula.tokenizer import Token
from openpyxl.worksheet.datavalidation import expand_cell_ranges
from sympy.parsing.sympy_parser import parse_expr
import re

def parse_formula(formula):
    """
    Returns the Sympy-parsed form for of the passed formula.
    So far, this method will remove the initial equals (=) sign and expand any cell ranges.
    """
    expanded_form = ""     
    formula_tokenizer = Tokenizer(formula)
    
    for token in formula_tokenizer.items:
        if token.subtype == Token.RANGE: # TODO: This will still fail on sheet reference ranges.
            expanded_range = expand_cell_ranges(token.value)
            encoded_references = [encode_cell_reference(r) for r in expanded_range]
            expanded_form += ",".join(encoded_references)
        else:
            expanded_form += token.value
    
    print(f"Input formula: {formula}")
    print(f"Expanded formula: {expanded_form}")
    
    return parse_expr(expanded_form)

def encode_cell_reference(reference):
    """
    Encodes the string of cell reference so it won't clash with predefined variables in Sympy.
    """
    
    column_finder = re.search('(.+?)[0-9]+', reference)
    
    # Early return
    if not column_finder:
        return reference
    
    column = column_finder.group(1)
    row = reference.replace(column, "")
    
    return f"{column}_{row}"
    
def decode_cell_reference(encoded_reference):
    """
    Decodes the cell reference from encode_cell_reference method so it can be used in normal spreadsheet operations.
    """
    return encoded_reference.replace('_','')
    
    

In [24]:
### Sample reference markers

import re

cell_reference = 'ABC1234'
m = re.search('(.+?)[0-9]+', cell_reference)

if m:
    found = m.group(1)
    print(f"found: {found}")
else:
    print("nothing found.")
    
encoded_reference = encode_cell_reference(cell_reference)
decoded_reference = decode_cell_reference(encoded_reference)

print(f"Encoded reference: {encoded_reference}")
print(f"Decoded reference: {decoded_reference}")


found: ABC
Encoded reference: ABC_1234
Decoded reference: ABC1234


In [25]:
from sympy import simplify

# Test cases of the function above.

# These values doesn't work:
# test_formulas = ["5", "=A1", "=SUM(A1,D1)", "=SUM($A$1, D1:F5)", "=SUM(SUM($A$1:$D$2), SUM(E1:G7))"]
# test_formulas = ["5", "=A1", "=SUM(A1,D1)", "=SUM($A$1, D1:F5)"]

# Huh. Perhaps Sympy has limits on the number of parameter that can be passed to a function?
# Yes, it still works when it was A1:D2... and failed on A1:E2.
# OOOOH WAIT! IT FAILED BECAUSE OF E1, WHICH WAS THOUGHT AS AN EULER'S NUMBER!
test_formulas = ["5", "=A1", "=SUM(A1,D1)", "=SUM(A1:E2)"]
parsed_formulas = [parse_formula(f) for f in test_formulas]


print("Test formulas:")
print(test_formulas)
print("\nParsed formulas:")
print(parsed_formulas)

for formula in parsed_formulas:
    is_equal = simplify(formula - formula) == 0
    print(f"Formula: {formula}")
    print(f"Is equal? {is_equal}")

Input formula: 5
Expanded formula: 5
Input formula: =A1
Expanded formula: A_1
Input formula: =SUM(A1,D1)
Expanded formula: SUM(A_1,D_1)
Input formula: =SUM(A1:E2)
Expanded formula: SUM(E_2,C_2,B_1,C_1,D_1,B_2,A_2,A_1,D_2,E_1)
Test formulas:
['5', '=A1', '=SUM(A1,D1)', '=SUM(A1:E2)']

Parsed formulas:
[5, A_1, SUM(A_1, D_1), SUM(E_2, C_2, B_1, C_1, D_1, B_2, A_2, A_1, D_2, E_1)]
Formula: 5
Is equal? True
Formula: A_1
Is equal? True
Formula: SUM(A_1, D_1)
Is equal? True
Formula: SUM(E_2, C_2, B_1, C_1, D_1, B_2, A_2, A_1, D_2, E_1)
Is equal? True
